In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
plats = "/content/terminos_contexto_generado_inst_1shot_limpio.csv"
jap = "/content/eurovoc_contexto_generado_llama8b_inst_3shot_limpio.csv"
gold="/content/gold_standard_enlazado.csv"

In [ ]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
pip install sentence-transformers pandas scikit-learn


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch


# Cargar el modelo
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Leer los archivos CSV
plats_df = pd.read_csv(plats)
jap_df = pd.read_csv(jap)
gold_standard = pd.read_csv(gold)

# Limpiar nombres de columnas
plats_df.columns = plats_df.columns.str.strip()
jap_df.columns = jap_df.columns.str.strip()
gold_standard.columns = gold_standard.columns.str.strip()

# Concatenar 'Término' + 'Ventana_de_contexto' como input para embeddings
plats_sentences = plats_df['Termino'].astype(str) + "," + plats_df['Ventana_de_contexto'].astype(str)
jap_sentences = jap_df['Término'].astype(str) + "," + jap_df['Ventana_de_contexto'].astype(str)

# Obtener embeddings
plats_embeddings = model.encode(plats_sentences.tolist(), convert_to_tensor=True)
jap_embeddings = model.encode(jap_sentences.tolist(), convert_to_tensor=True)

# Normalizar los embeddings para similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Umbral de similitud
umbral = 0.8

# Máscara: dónde la similitud supera el umbral
mask = cosine_similarities > umbral
indices = torch.nonzero(mask, as_tuple=False)

# Almacenar matches detectados
detected_matches = []

# Comparar términos
for index in indices:
    i, j = index[0].item(), index[1].item()
    similarity = cosine_similarities[i, j].item()
    plats_term = plats_df['Termino'].iloc[i]
    jap_term = jap_df['Término'].iloc[j]

    detected_matches.append((plats_term, jap_term))

    print(f"Match ({similarity:.4f}): '{plats_term}' ↔ '{jap_term}'\n")

# -----------------------
# Evaluación con gold-standard
# -----------------------
gold_pairs = set(zip(gold_standard['Termino_1'], gold_standard['Término_2']))
detected_pairs = set(detected_matches)

# y_true: 1 si está en gold, 0 si no
y_true = [1 if pair in gold_pairs else 0 for pair in detected_pairs]
y_pred = [1] * len(detected_pairs)

# Para recall: añadir los del gold que no se detectaron
for pair in gold_pairs:
    if pair not in detected_pairs:
        y_true.append(1)
        y_pred.append(0)

from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print("\n--- Métricas de Evaluación ---")
print(f"Matches detectados: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Match (0.9064): 'trabajo nocturno' ↔ 'trabajo nocturno'

Match (0.9439): 'maternidad' ↔ 'maternidad'

Match (0.8130): 'remuneración' ↔ 'remuneración del trabajo'

Match (0.8065): 'contrato' ↔ 'contrato de trabajo'

Match (0.8006): 'riesgo' ↔ 'trabajo nocturno'

Match (0.8984): 'riesgo' ↔ 'riesgo cubierto'

Match (0.8049): 'jornada' ↔ 'jornada de trabajo'

Match (0.8887): 'jornada' ↔ 'jornada de trabajo'

Match (0.8187): 'jornada' ↔ 'horario de trabajo'

Match (0.8068): 'horario flexible' ↔ 'trabajo nocturno'

Match (0.8225): 'votación' ↔ 'votación'

Match (0.8931): 'contrato de trabajo' ↔ 'contrato de trabajo'

Match (0.8768): 'organización del trabajo' ↔ 'organización del trabajo'

Match (0.8350): 'trabajo a distancia' ↔ 'trabajo nocturno'

Match (0.8406): 'consanguinidad' ↔ 'parentesco'

Match (0.9755): 'salud pública' ↔ 'salud pública'

Match (0.8049): 'jurisdicción social' ↔ 'jurisdicción social'

Match (0.8104): 'religión' ↔ 'religión'

Match (0.8288): 'salario mínimo interprofesi

In [ ]:
# Evaluar contra el gold standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set(detected_matches)

# Calcular los aciertos
true_positives = gold_pairs.intersection(detected_pairs)
false_positives = detected_pairs - gold_pairs
false_negatives = gold_pairs - detected_pairs

# Métricas de evaluación
precision = len(true_positives) / (len(true_positives) + len(false_positives)) if (len(true_positives) + len(false_positives)) > 0 else 0
recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if (len(true_positives) + len(false_negatives)) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nResultados de la evaluación con el gold standard:")
print(f"True Positives (Aciertos): {len(true_positives)}")
print(f"False Positives (Falsos Positivos): {len(false_positives)}")
print(f"False Negatives (Falsos Negativos): {len(false_negatives)}")
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")


Resultados de la evaluación con el gold standard:
True Positives (Aciertos): 0
False Positives (Falsos Positivos): 29
False Negatives (Falsos Negativos): 9
Precisión: 0.0000
Recall: 0.0000
F1-Score: 0.0000
